In [2]:
stop_word = pd.read_csv('停用词.csv',names=['w'],sep='\t',encoding='utf-8')
l=[w for w in jieba.cut('西安兵马俑') if w not in list(stop_word.w)]
for i in l:
    print(i,end='/')

西安/兵马俑/

In [3]:
import pandas as pd
import jieba
gp = open("评论文本.txt",encoding='utf-8').read()
gp = gp.replace('\n','')
gp = gp.replace(' ','')
word_list = [w for w in jieba.cut(gp) if w not in list(stop_word.w)] # 去除停用词

# 转为数据框，方便统计词频
df = pd.DataFrame(word_list, columns = ['word'])
#df.head()

result = df.groupby(['word']).size()
# print(result)
freqlist = result.sort_values(ascending=False) # 降序
freqlist[:20]

word
值得     8521
西安     6951
历史     6130
导游     5343
城墙     4321
兵马俑    3525
博物馆    2967
景色     2958
震撼     2766
小时     2626
华山     2607
排队     2350
携程     2255
风景     2234
感受     2124
晚上     2120
体验     2119
坐      2099
表演     2090
喜欢     2043
dtype: int64

In [7]:
from sklearn.decomposition import LatentDirichletAllocation
# 指定 lda 主题数
n_topics = 5
lda = LatentDirichletAllocation(
    n_components=n_topics, max_iter=50,
    learning_method='online',
    learning_offset=50.,
    random_state=0)
print(lda)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=50, n_components=5, random_state=0)


In [16]:
# 去重、去缺失、分词
df1=df.drop_duplicates().reset_index().drop('index',axis=1)
df2=df1.iloc[:30000,:]
df2

,word
0,遇
1,好人
2,强
3,汉中
4,西安
...,...
29995,院是
29996,迈出
29997,之西峻险
29998,挺秀


In [17]:
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
import os
import re
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer

# 构造 TF-IDF
tf_idf_vectorizer = TfidfVectorizer()
tf_idf = tf_idf_vectorizer.fit_transform(df2.word)
# 特征词列表
feature_names = tf_idf_vectorizer.get_feature_names()
# 特征词 TF-IDF 矩阵
matrix = tf_idf.toarray()
# 指定 lda 主题数
n_topics = 5
lda = LatentDirichletAllocation(
    n_components=n_topics, max_iter=50,
    learning_method='online',
    learning_offset=50.,
    random_state=0)
# 核心，给 LDA 喂生成的 TF-IDF 矩阵
lda.fit(tf_idf)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=50, n_components=5, random_state=0)

In [19]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd


def top_words_data_frame(model: LatentDirichletAllocation,
                         tf_idf_vectorizer: TfidfVectorizer,
                         n_top_words: int) -> pd.DataFrame:
    '''
    求出每个主题的前 n_top_words 个词

    Parameters
    ----------
    model : sklearn 的 LatentDirichletAllocation 
    tf_idf_vectorizer : sklearn 的 TfidfVectorizer
    n_top_words :前 n_top_words 个主题词

    Return
    ------
    DataFrame: 包含主题词分布情况
    '''
    rows = []
    feature_names = tf_idf_vectorizer.get_feature_names()
    for topic in model.components_:
        top_words = [feature_names[i]
                     for i in topic.argsort()[:-n_top_words - 1:-1]]
        rows.append(top_words)
    columns = [f'topic {i+1}' for i in range(n_top_words)]
    df = pd.DataFrame(rows, columns=columns)

    return df


def predict_to_data_frame(model: LatentDirichletAllocation, X: np.ndarray) -> pd.DataFrame:
    '''
    求出文档主题概率分布情况

    Parameters
    ----------
    model : sklearn 的 LatentDirichletAllocation 
    X : 词向量矩阵

    Return
    ------
    DataFrame: 包含主题词分布情况
    '''
    # 求出给定文档的主题概率分布矩阵
    matrix = model.transform(X)
    columns = [f'P(topic {i+1})' for i in range(len(model.components_))]
    df = pd.DataFrame(matrix, columns=columns)
    return df

In [21]:

# 本地 csv 文件，可以是本地文件，也可以是远程文件
source_csv_path = 'answers.csv'
# 文本 csv 文件里面文本所处的列名,注意这里一定要填对，要不然会报错的！
document_column_name = '回答内容'
# 输出主题词的文件路径
top_words_csv_path = 'top-topic-words.csv'
# 输出各文档所属主题的文件路径
predict_topic_csv_path = 'document-distribution.csv'
# 可视化 html 文件路径
html_path = 'document-lda-visualization.html'
# 选定的主题数
n_topics = 5
# 要输出的每个主题的前 n_top_words 个主题词数
n_top_words = 20

# 计算 n_top_words 个主题词
top_words_df = top_words_data_frame(lda, tf_idf_vectorizer, n_top_words)

# 保存 n_top_words 个主题词到 csv 文件中
top_words_df.to_csv(top_words_csv_path, encoding='utf-8-sig', index=None)

# 转 tf_idf 为数组，以便后面使用它来对文本主题概率分布进行计算
X = tf_idf.toarray()

# 计算完毕主题概率分布情况
predict_df = predict_to_data_frame(lda, X)

# 保存文本主题概率分布到 csv 文件中
predict_df.to_csv(predict_topic_csv_path, encoding='utf-8-sig', index=None)

https://zhuanlan.zhihu.com/p/403058948